In [ ]:
import os
import sklearn
import tensorflow
import matplotlib.pyplot as plt
from sklearn import * # Using wildcard (*) as a place holder
from scipy import * # Using wildcard (*) as a place holder


def main():
    pass


if __name__ == "__main__":
    main()


main
